<a href="https://colab.research.google.com/github/thekishanpatel/Projects/blob/master/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentiment Analysis**

In an age where media is being critizied, I made a simple sentiment analysis tool that grabs headlines from NYT and Reuters based on a certain topic, and then uses pre-trained classifiers to give a 'sentiment' score on the headlines. The idea was to see if one or both of the news-sources have a bias for or against the topic. 

This was merely a practice for me to learn using NLP libraries and applying Sentiment Analysis, something I wanted to get comfortable with prior to applying it to my larger project. 

# **Import Libraries**

In [0]:
from textblob import TextBlob
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import re as re
%matplotlib inline

import requests
from bs4 import BeautifulSoup

# **Get Topic From User**

In [32]:
inp = input('Enter topic you want to search:')
inp = re.sub(' ', '+', inp)

Enter topic you want to search:bernie sanders


# **Get the Headlines from the Two Sites**

In [0]:
nytu = 'https://www.nytimes.com/search?query=' + inp
nytuparent = 'https://www.nytimes.com'
pge = requests.get(nytu)
soup = BeautifulSoup(pge.content, 'html.parser')
wl = soup.findAll('div', attrs={'class':'css-e1lvw9'})
headlines = []
links = []
for a in wl:
  hl = a.find('h4', attrs={'class':'css-2fgx4k'})
  link = a.find('a')
  headlines.append(hl.get_text())
  link = nytuparent + link.get('href')
  links.append(link)
nytpd = pd.DataFrame({'Headlines' : headlines, 'Links' : links})

In [0]:
ru = 'https://www.reuters.com/search/news?blob=' + inp
ruparent = 'https://www.reuters.com'
pge = requests.get(ru)
soup = BeautifulSoup(pge.content, 'html.parser')
wl1 = soup.findAll('div', attrs={'class':'search-result-content'})
headlines = []
links = []
for a in wl1:
  hl = a.find('h3', attrs={'class':'search-result-title'})
  link = a.find('a')
  link = ruparent + link.get('href')
  headlines.append(hl.get_text())
  links.append(link)
reutpd = pd.DataFrame({'Headlines' : headlines, 'Links' : links})

# **Build Classifier**

In [17]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [0]:
def sentiment(data):
  senti = SentimentIntensityAnalyzer();
  hl = data['Headlines'].values
  n = len(hl)
  Score = []

  for i in range(0, n, 1):
    s = senti.polarity_scores(hl[i])
    Score.append(s['compound'])
  return Score

In [0]:
def sentiment2(data):
  hl = data['Headlines'].values
  n = len(hl)
  Score = []

  for i in range(0, n, 1):
    s = TextBlob(hl[i])
    Score.append(s.sentiment.polarity)
  return Score

In [0]:
def comment(data):
  avg = data['Average Score'].values; num = len(avg)
  comment = []; p = 'Positive'; n = 'Negative'; neu = 'Neutral';
  
  for i in range(0, num, 1):
    if avg[i] > 0.05: 
      comment.append(p)
    elif (-0.05 <= avg[i] <= 0.05):
      comment.append(neu)
    else:
      comment.append(n)
  return comment

In [0]:
nytScore = sentiment(nytpd); nytpd.insert(1,"Score",nytScore)
reutScore = sentiment(reutpd); reutpd.insert(1,"Score",reutScore)

nytScore2 = sentiment2(nytpd); nytpd.insert(2,"Score2",nytScore2)
reutScore2 = sentiment2(reutpd); reutpd.insert(2,"Score2",reutScore2)

In [0]:
nytpd.insert(3,'Average Score', (nytpd.Score+nytpd.Score2)/2)
reutpd.insert(3,'Average Score', (reutpd.Score+reutpd.Score2)/2)

In [0]:
nytcmt = comment(nytpd); nytpd.insert(4, 'Review', nytcmt)
reutcmt = comment(reutpd); reutpd.insert(4, 'Review', reutcmt)

# **Results**

In [38]:
nytpd

,Headlines,Score,Score2,Average Score,Review,Links
0,The Scent of Bernie Sanders,0.0000,0.0,0.00000,Neutral,https://www.nytimes.com/2020/01/31/style/berni...
1,Bernie Sanders Can’t Win,0.5859,0.8,0.69295,Positive,https://www.nytimes.com/2020/01/31/opinion/ber...
2,Joe Biden Attacks Bernie Sanders’s Record on Guns,-0.4404,0.0,-0.22020,Negative,https://www.nytimes.com/2020/02/07/us/politics...
3,Bernie Sanders’s Surge Owes a Lot to Voters of...,0.0000,0.0,0.00000,Neutral,https://www.nytimes.com/2020/01/31/us/politics...
4,President Bernie Sanders,0.0000,0.0,0.00000,Neutral,https://www.nytimes.com/2020/01/16/opinion/ber...
5,The Bernie Sanders Fallacy,0.0000,0.0,0.00000,Neutral,https://www.nytimes.com/2020/01/16/opinion/the...
6,What Are Iowa’s Satellite Caucuses? And How Di...,0.4696,0.0,0.23480,Positive,https://www.nytimes.com/2020/02/06/us/politics...
7,Please Stop Calling Bernie Sanders a Populist,0.0258,0.0,0.01290,Neutral,https://www.nytimes.com/2020/01/23/opinion/ber...
8,Hillary Clinton Slams Bernie Sanders for Not W...,0.0000,0.0,0.00000,Neutral,https://www.nytimes.com/2020/01/31/us/politics...
9,"Bernie Sanders, and How Indian Food Can Predic...",0.0000,0.0,0.00000,Neutral,https://www.nytimes.com/2020/01/30/upshot/bern...


In [39]:
reutpd

,Headlines,Score,Score2,Average Score,Review,Links
0,Breakingviews - Cox: Who’s afraid of a Preside...,0.0000,-0.6,-0.30000,Negative,https://www.reuters.com/article/us-usa-politic...
1,Bernie Sanders to defend democratic socialism ...,0.0000,0.0,0.00000,Neutral,https://www.reuters.com/article/idUSKCN1TC2XY
2,Bernie Sanders accuses Walmart of paying 'star...,-0.3400,0.0,-0.17000,Negative,https://www.reuters.com/article/idUSL2N23C0W9
3,Bernie Sanders urges Walmart to boost 'starvat...,0.4019,0.0,0.20095,Positive,https://www.reuters.com/article/idUSKCN1T622J
4,Bernie Sanders to defend democratic socialism ...,0.0000,0.0,0.00000,Neutral,https://www.reuters.com/article/idUSL2N23I1Y3
5,"Bernie Sanders 'up and about,' will be in next...",0.0000,0.0,0.00000,Neutral,https://www.reuters.com/article/idUSKBN1WI21E
6,"After feud with Warren, Bernie Sanders release...",-0.3400,0.0,-0.17000,Negative,https://www.reuters.com/article/idUSL1N29M10V
7,Florida man gets 15 months for threatening to ...,-0.8442,0.0,-0.42210,Negative,https://www.reuters.com/article/idUSKCN1VD2AG
8,'Nobody likes him' - Hillary Clinton bashes Be...,0.4215,0.0,0.21075,Positive,https://www.reuters.com/article/idUSL1N29Q0LM
9,UPDATE 2-'Nobody likes him' - Hillary Clinton ...,0.4215,0.0,0.21075,Positive,https://www.reuters.com/article/idUSL1N29Q0WK
